In [1]:
!pip install Office365-REST-Python-Client
!pip install office365
!pip install awswrangler

     |████████████████████████████████| 820 kB 24.4 MB/s            
     |████████████████████████████████| 89 kB 11.3 MB/s            
     |████████████████████████████████| 47 kB 823 kB/s             
     |████████████████████████████████| 163 kB 56.7 MB/s            
     |████████████████████████████████| 377 kB 73.2 MB/s            
     |████████████████████████████████| 178 kB 76.1 MB/s            
     |████████████████████████████████| 85 kB 1.0 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 5.6 MB 30.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 388 kB 72.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 232 kB 74.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 125 kB 73.4 MB/s            
     |████████████████████████████████| 59 kB 9.7 MB/s             
  Preparing 

     |████████████████████████████████| 131 kB 77.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 41 kB 127 kB/s             
     |████████████████████████████████| 151 kB 76.6 MB/s            
     |████████████████████████████████| 70 kB 2.1 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 26.9 MB 41.7 MB/s            
     |████████████████████████████████| 340 kB 72.6 MB/s            
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3576 sha256=fc9393893b7e99cc52962a66abe7fe1b3e030be1f8c5858f6131ebe8c80cb058
  Stored in directory: /home/ec2-user/.cache/pip/wheels/39/ed/1a/d1fbf258dd3b80cf68bdcc602c60f46039c4aacd9ae672b827
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=c8e419deaa6eca97f8080ac013ad3146fa4be4da38b22fe7f29b88f65ca6f6a2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/19/f5/6d/a97dd4f22376d4472d5f4c76

     |████████████████████████████████| 226 kB 24.1 MB/s            
     |████████████████████████████████| 20.1 MB 57.0 MB/s            
     |████████████████████████████████| 43 kB 1.0 MB/s             
     |████████████████████████████████| 112 kB 74.6 MB/s            
     |████████████████████████████████| 207 kB 76.5 MB/s            
     |████████████████████████████████| 6.6 MB 70.1 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: lxml
    Found existing installation: lxml 4.6.4
    Uninstalling lxml-4.6.4:
      Successfully uninstalled lxml-4.6.4


In [113]:
import imp
import logging
import boto3
import os
import logging
from botocore.exceptions import ClientError
import json
import requests
import csv
import struct

from subprocess import call
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File

from datetime import datetime, timedelta
import awswrangler as wr
import pandas as pd
import numpy as np

In [114]:
# base_excepciones_banderas_rojas.xlsx
s3_client = boto3.client('s3')

## cargamos todas las claves de acceso desde las variables de entorno
sharepoint_url = "https://bancoink.sharepoint.com/sites/data2022/"
client_id = "a7bc6720-474d-4253-b3ff-6fa0e36de593"
DECRYPTED = "9Qxc6nz6yZ/AR4z7eu0sXEtlBe84IuoGg5F5BEzxsqc="

folder_url = "/sites/data2022/Documentos%20compartidos%2Fencaje_project/"
bucket_name = 'cnk-datalake' 
file_name = 'Historico_arqueos_data.xlsx'

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [115]:
day_scheduled=datetime.today()-timedelta(days=10)

In [116]:
days_num="'"+str(10)+"'"

In [117]:
context_auth = AuthenticationContext(sharepoint_url)
context_auth.acquire_token_for_app(client_id, DECRYPTED)

ctx = ClientContext(sharepoint_url, context_auth)
web = ctx.web
ctx.load(web)
ctx.execute_query()

path_file = folder_url + file_name 

response = File.open_binary(ctx, path_file)
#

path_file_lambda_xlsx = file_name

path_file_lambda_csv = "archivo.csv"

with open(path_file_lambda_xlsx, "wb") as local_file:
    local_file.write(response.content)
    print("Archivo xlsx descargado....")

day_schedule=datetime.today()-timedelta(days=int(days_num.replace("'","")))
    
## calling the data from sharepoint and leaving just the last 10 days
data=pd.read_excel(path_file_lambda_xlsx,engine="openpyxl")

Archivo xlsx descargado....


In [118]:
string_query=f"""
SELECT so."service_order_id",so."service_order_status",
sos."service_order_status_name",so."service_order_updated_date",
so."service_order_created_date",so."service_order_piggybank_id",
so."service_order_type",sot."service_type_name",so."service_order_sequence",
so."service_order_canceled_by",so."service_order_canceled_date",
so."service_order_door_opened_date",so."service_order_door_closed_date",
sod."value",sod."created_at",pgb."piggy_bank_operational_account",
cast(json_extract(sod."detail",'$[0].quantity') as Integer) as "50_amount",
cast(json_extract(sod."detail",'$[1].quantity') as Integer) as "100_amount",
cast(json_extract(sod."detail",'$[2].quantity') as Integer) as "200_amount",
cast(json_extract(sod."detail",'$[3].quantity') as Integer) as "500_amount",
cast(json_extract(sod."detail",'$[4].quantity') as Integer) as "1000_amount",
gc."maplocation_correct"
FROM "piggybank"."service_orders" as so LEFT JOIN 
"piggybank"."service_order_details" as sod
ON so."service_order_id"=sod."service_order_id"
LEFT JOIN "AwsDataCatalog"."cnk_datalake"."ow_etl_hc_geolocation" as gc
ON so."service_order_piggybank_id"=gc."piggy_bank_id"
LEFT JOIN "piggybank"."service_order_status" sos
ON so."service_order_status"=sos."service_order_status_id"
LEFT JOIN "piggybank"."service_types" sot
ON so."service_order_type"=sot."service_type_id"
LEFT JOIN "piggybank"."piggy_bank" pgb
ON so."service_order_piggybank_id"=pgb."piggy_bank_id"
where so."service_order_created_date">cast(current_date - interval {days_num} day as timestamp)
AND so."service_order_id" not in (SELECT distinct("service_order_id") FROM "AwsDataCatalog"."cnk_datalake"."arqueo_conciliados");
"""

data_aurora=wr.athena.read_sql_query(string_query,
                                     database="piggybank",
                                     data_source="aurora_coink",
                                     ctas_approach=False,
                                     s3_output='s3://cnk-datalake/experimentos001/athena_datawrangler/encaje/')

In [119]:
## Just leave the service_orders that corresponds with "arqueos"
data_aurora=data_aurora[data_aurora["service_type_name"]=="VAULT_ACCESS"]

## leave only the service_orders that were finished already
data_aurora=data_aurora[(data_aurora["service_order_status_name"]=="FINISHED") | (data_aurora["service_order_status_name"]=="DOOR_CLOSED") | (data_aurora["service_order_status_name"]=="DOOR_OPENED") | (data_aurora["service_order_status_name"]=="FINISHED_FROM_OPERATOR")]

In [120]:
## transform the data to the same type to merge it
data_aurora["service_order_sequence"]=data_aurora["service_order_sequence"].astype(int)
data["service_order_sequence"]=data["service_order_sequence"].astype(int)

In [121]:
## confirm that there are not service orders without a match in total_details.
data_aurora=data_aurora[~(data_aurora['value'].isna())]

In [122]:
## bring the data from the forms to compare if the data has been sent previously
string_query_1=f"""
SELECT * FROM "AwsDataCatalog"."cnk_datalake"."arqueo_conciliados"
WHERE cast("fecha_de_envio_formulario" as timestamp)>cast(current_date - interval {days_num} day as timestamp);
"""
data_aurora_form=wr.athena.read_sql_query(string_query_1,
                                     database="cnk_datalake",
                                     data_source="AwsDataCatalog",
                                     s3_output='s3://cnk-datalake/experimentos001/athena_datawrangler/encaje_3/')

## create a column so we can identify the registers that have already been sent
data_aurora_form["merged"]=1

In [123]:
data_aurora_form

,service_order_id,nombre_operario,empresa,email,ciudad,maplocation,fecha,es_reemplazo,solucionado,fecha_de_envio_formulario,...,1000_amount,50_coin_amount,100_coin_amount,200_coin_amount,500_coin_amount,1000_coin_amount,piggy_bank_operational_account,core_amount,value_difference_core_vs_arqueo,merged
0,53917d56-7461-4f62-906f-17c77f6ae157,Juan Carlos Oviedo,RAPICAMBIOS,luisenri.valderrama@gmail.com,SANTANDER,CC Cañaveral - Segundo Piso,2023-03-13,no,no,2023-03-14 22:55:46,...,1557,1662.0,4342.0,6482.0,3900.0,1556.0,0000000256,5314700.0,-5000.0,1
1,1209dc88-884e-457d-bde6-67ae221a33cc,juan alvarez,RAPICAMBIOS,luisenri.valderrama@gmail.com,BOGOTÁ,CC Plaza Imperial - Plaza Trombos,2023-03-14,no,no,2023-03-14 23:32:17,...,1062,1.0,2824.0,3700.0,3680.0,1064.0,0000000164,3930500.0,4050.0,1
2,831fd9e2-c135-4d0c-a897-0492f32e2f44,Juan Alvarez,RAPICAMBIOS,luisenri.valderrama@gmail.com,BOGOTÁ,CC Unicentro - Salida Peatonal 8,2023-03-14,no,no,2023-03-14 23:34:25,...,747,0.0,2036.0,2154.0,1721.0,742.0,0000000183,2239700.0,2800.0,1
3,76fa0724-29e5-4cdd-a7a1-95e818ec66ba,Marcelo Mejia,RAPICAMBIOS,luisenri.valderrama@gmail.com,BOGOTÁ,CC Plaza de las Américas - Plaza Central,2023-03-13,no,no,2023-03-14 23:22:55,...,3991,1563.0,5117.0,6706.0,6737.0,3986.0,0000000060,9285300.0,-250.0,1
4,967b86f3-f5d8-40f6-995c-8c680c815422,juan alvarez,RAPICAMBIOS,luisenri.valderrama@gmail.com,BOGOTÁ,Universidad de los Andes - ML Piso 2,2023-03-14,no,no,2023-03-14 23:39:28,...,420,617.0,1834.0,2226.0,1174.0,415.0,0000000013,1661250.0,-200.0,1
5,1bfcef53-904d-43dc-bd72-d294699900b2,juan carlos oviedo,RAPICAMBIOS,luisenri.valderrama@gmail.com,SANTANDER,CC Megamall - Primer Piso,2023-03-13,no,no,2023-03-14 23:18:40,...,1124,1390.0,3800.0,5182.0,4046.0,1121.0,0000000229,4653250.0,23350.0,1
6,cdc36a4f-fc17-4a77-acf6-dfa0b4baee2d,Juan alvarez,RAPICAMBIOS,luisenri.valderrama@gmail.com,BOGOTÁ,TM Transmilenio Portal Américas - Ingreso Peat...,2023-03-14,no,no,2023-03-14 23:29:59,...,195,427.0,1223.0,2247.0,769.0,195.0,0000000205,1208900.0,36350.0,1
7,a67d671b-a11c-4cb7-822a-4fb654de804a,Juan Carlos Oviedo,RAPICAMBIOS,luisenri.valderrama@gmail.com,SANTANDER,CC La Florida - Primer Piso,2023-03-13,no,no,2023-03-14 22:53:16,...,493,613.0,2746.0,2251.0,2171.0,495.0,0000000257,2334450.0,-1500.0,1
8,a5fbeb43-80a0-486f-b697-cb61aabe8713,juan alvarez,RAPICAMBIOS,luisenri.valderrama@gmail.com,BOGOTÁ,Universidad de los Andes - LL Piso 2,2023-03-14,no,no,2023-03-14 23:37:13,...,553,799.0,2102.0,2366.0,1626.0,541.0,0000000015,2076600.0,-750.0,1
9,99a5c932-2080-4da0-a665-ffb626b99387,juan Carlos Oviedo,RAPICAMBIOS,luisenri.valderrama@gmail.com,SANTANDER,CC Cuarta Etapa,2023-03-13,no,no,2023-03-14 23:08:57,...,838,558.0,2141.0,2973.0,2472.0,831.0,0000000228,2903650.0,50.0,1


In [124]:
## transform the datetime to the bogota time
data_aurora["service_order_created_date"]=data_aurora["service_order_created_date"]-timedelta(hours=5, minutes=0)

## transform the data from the back to date without hours,minutes, etc
data_aurora["fecha"]=[i.date() for i in data_aurora["service_order_created_date"]]
data["fecha"]=[i.date() for i in data["fecha"]]
#data_aurora_form['fecha']=[i.date() for i in data_aurora_form["fecha"]]

In [125]:
## see which registers are in both tables
data=pd.merge(data,data_aurora_form[["service_order_sequence","merged","fecha","maplocation","solucionado"]],
              on=["fecha","maplocation","service_order_sequence"],
              how="left")
data["merged"].fillna(0,inplace=True)

## left just the registers that have't been sent previously
data=data[(data["merged"]!=1) & ((data["solucionado"]=="no") | (data["solucionado"].isna())) | ((data["solucionado"]=="no") & (data["es_reemplazo"]=="si"))]

In [127]:
## format the columns as string
data["nombre_operario"]=[str(i) for i in data["nombre_operario"]]
data["email"]=[str(i) for i in data["email"]]
data["comentarios"]=[str(i) for i in data["comentarios"]]

In [128]:
if data.empty:
    print("no forms data without match")
else:
    ## load all data to s3
    path_file="s3://cnk-datalake/piggybank/arqueo_project/formulario/excel_arqueo.parquet"

    ## save the data in s3
    wr.s3.to_parquet(
                    df=data,
                    path=path_file,
                    dataset=False)

In [129]:
## filter only the last 10 days
data=data[data['fecha']>=day_scheduled.date()]

In [130]:
## rename maplocation columns so we can do the merge with the next table
data_aurora.rename(columns={'maplocation_correct':'maplocation'},inplace=True)

In [131]:
## merging the data from the sharepoint with aurora data
data_merged=pd.merge(data,data_aurora,on=["fecha","maplocation","service_order_sequence"],how="outer")

In [132]:
# diference in pesos from the back and the form submitted by the company
data_merged["value_difference_back_arqueo"]=data_merged["value"]-data_merged["monto total encontrado"]

## rename the columns for the understanding
data_merged.rename(columns={"value":"monto_back","monto total encontrado":"monto_arqueo"},inplace=True)

In [133]:
## select just the columns that we want to use
data_wo_col=data_merged[['nombre_operario', 'empresa',
       'ciudad', 'maplocation', 'fecha',
       'fecha de envio formulario', 'comentarios','service_order_sequence',
       'service_order_updated_date', 'service_order_created_date',
       'service_order_door_opened_date', 'service_order_door_closed_date','monto_arqueo',
       'monto_back','value_difference_back_arqueo',"piggy_bank_operational_account","merged"]]

## data to storei in the excel
data_merged=data_merged[['service_order_id','nombre_operario', 'empresa', 'email',
       'ciudad', 'maplocation', 'fecha','es_reemplazo','solucionado',
       'fecha de envio formulario', 'comentarios','service_order_sequence',
       'service_order_updated_date', 'service_order_created_date',
       'service_order_door_opened_date', 'service_order_door_closed_date','monto_arqueo',
       'monto_back','value_difference_back_arqueo',
       '50_amount', '100_amount', '200_amount','500_amount', '1000_amount',
       '50_coin_amount', '100_coin_amount','200_coin_amount', '500_coin_amount', '1000_coin_amount','piggy_bank_operational_account']]

In [134]:
## load the data that already mergedto a route in s3
data_merged_inner=data_merged[(~((data_merged["empresa"].isna())|(data_merged["service_order_id"].isna()))) | ((data_merged["es_reemplazo"]=="si")&((data_merged["solucionado"]=="no")))]

In [137]:
## function to load row by row the data to s3
def funct_load_data(data_inner,data_columns):
    temp=pd.DataFrame(data=[data_inner],columns=data_columns)
    path_file="s3://cnk-datalake/piggybank/arqueo_project/conciliados/"+str(temp["fecha"].tolist()[0]).replace(" ","_")+"_"+str(temp["maplocation"].tolist()[0]).replace(" ","_")+"_"+str(temp["service_order_sequence"].tolist()[0]).replace(" ","_")
    
    if temp["es_reemplazo"].tolist()[0]=="no":
        
        temp['solucionado']='no'
        ## save each row into s3
        wr.s3.to_parquet(
                df=temp,
                path=path_file,
                dataset=False
            )
    elif (temp["es_reemplazo"].tolist()[0]=="si") & (temp["solucionado"].tolist()[0]=="no"):
        
        ## bring the log that corresponds with the observation to replace
        temp_1=wr.s3.read_parquet(path_file)
        
        for i in ['nombre_operario','empresa','email','ciudad','es_reemplazo','50_coin_amount',
           '100_coin_amount', '200_coin_amount', '500_coin_amount',
           '1000_coin_amount', 'monto_arqueo',
           'fecha de envio formulario', 'comentarios', 'solucionado']:
            
            temp_1[i]=temp[i]
        
        ## updating the database so it knows that the log corresponds with a replacement
        temp_1['solucionado']='si'
        
        ## agregar aca que recalcule la diferencia entre back y arqueo
        temp_1['value_difference_back_arqueo']=temp_1['monto_back']-temp_1['monto_arqueo']
        temp_1['value_difference_core_vs_arqueo']=temp_1['core_amount']-temp_1['monto_arqueo']
        
        ## copy the temp so we can load it into s3
        temp=temp_1[temp.columns.tolist()]
        
        ## save each row into s3
        wr.s3.to_parquet(
                df=temp,
                path=path_file,
                dataset=False)
        
    return list(temp.iloc[0,:])

In [55]:
if data_merged_inner.empty:
    table_1=pd.DataFrame(columns=["empresa","ciudad","fecha","maplocation",'core_amount',"monto_arqueo","monto_back","value_difference_back_arqueo",'value_difference_core_vs_arqueo',"service_order_sequence","piggy_bank_operational_account"])
else:
    # import last two service order logs for each piggybank
    string_query = '''
        SELECT 
            so."service_order_id",
            so."service_order_created_date",
            gc."maplocation_correct",
            gc."piggy_bank_id"
        FROM "piggybank"."service_orders" as so
        LEFT JOIN "AwsDataCatalog"."cnk_datalake"."ow_etl_hc_geolocation" as gc
        ON so."service_order_piggybank_id"= gc."piggy_bank_id"
        WHERE "service_order_status" in (5,6,7,9) and "service_order_type"= 2;
    '''

    sub_data_aurora=wr.athena.read_sql_query(string_query,
                                         database="piggybank",
                                         data_source="aurora_coink",
                                         ctas_approach=False,
                                         s3_output='s3://cnk-datalake/experimentos001/athena_datawrangler/encaje/')
    
    # sort logs values by the creation date of service date 
    sub_data_aurora = sub_data_aurora.sort_values(by=['service_order_created_date'])
    # change timezone from UTC to Bogota
    #sub_data_aurora['service_order_created_date'] = [(datetime.strptime(str(x)[0:18], '%Y-%m-%d %H:%M:%S') - timedelta(hours=5, minutes=0)) for x in sub_data_aurora['service_order_created_date']]
    sub_data_aurora['service_order_created_date']=sub_data_aurora['service_order_created_date']-timedelta(hours=5,minutes=0)
    # create a 1 step lag variable for each maplocation
    sub_data_aurora['shift_date'] = sub_data_aurora.groupby(['maplocation_correct']).service_order_created_date.shift(1)
    # take the difference between creation date and its lag
    sub_data_aurora['time_delta'] = sub_data_aurora['service_order_created_date'] - sub_data_aurora['shift_date']
    # subset the log data so that only the most recent log for each maplocation in the report table 1 are seen
    table_1_data = data_merged_inner[['service_order_created_date', 'maplocation']]
    #table_1_data['service_order_created_date'] =  [datetime.strptime(str(x)[0:18], '%Y-%m-%d %H:%M:%S') for x in table_1_data['service_order_created_date']]
    table_1_data = table_1_data.rename(columns={'maplocation':'maplocation_correct'})
    temp = pd.merge(sub_data_aurora, table_1_data, on=['service_order_created_date', 'maplocation_correct'], how='inner').drop_duplicates(subset=['service_order_created_date', 'maplocation_correct'])
    temp.shift_date = pd.to_datetime(temp.shift_date)
    
    # check if there are piggybanks which are having their first time of cash counting
    first_time = temp[temp.shift_date.isna()].piggy_bank_id.tolist()
    if len(first_time) > 0:
        # import piggybank creation date in database
        oinks_data = wr.s3.read_parquet(path='s3://cnk-datalake/maplocation/Output/etl_hc_geolocation/')
        # select variables of interest
        oinks_data = oinks_data[['created_date', 'piggy_bank_id']]
        # select piggybanks which are having their first time of cash counting
        first_time_oinks = oinks_data[oinks_data.piggy_bank_id.isin(first_time)]
        # select the minimum date between creation date vector and last time of cash counting vector
        start_date = min(temp.shift_date.min(),first_time_oinks.created_date.min())
        # select the date as start_date - 1 day
        date_str = datetime.strftime(start_date - timedelta(days=1), "%Y-%m-%d")
    # if there are no piggybanks which are having their first time of cash counting, use the minimum of the last time of cash counting vector
    else:  date_str = datetime.strftime(temp.shift_date.min() - timedelta(days=1), "%Y-%m-%d")
    date_str
    
    # import transactional data from piggybanks leaving only approved transactions
    # since the minimum date overall required piggybanks (date_str)
    # for only the requested piggybanks
    string_query = """SELECT 
    cr.transfer_id, 
    cr.amount,
    cr.creation_date,
    CAST(json_extract(cr.associated_data, '$.location_id') as varchar) as location_id,
    CAST(json_extract(cr.associated_data, '$.piggybank_id') as varchar) as piggybank_id
    FROM "core"."transfer_view" as cr
    WHERE CAST(json_extract(cr.associated_data, '$.piggybank_id') as varchar) in """ + "('"+ "', '".join(temp['piggy_bank_id'].tolist()) + "')" + "AND (CAST(cr.creation_date as date) > CAST('" + date_str + "' AS DATE)) AND cr.status_id = 2 ORDER BY CAST(cr.creation_date as date) ASC;"

    Athenadf = wr.athena.read_sql_query(string_query, 
                                         database="core",
                                         data_source="aurora_coink",
                                        ctas_approach=False,
                                        s3_output = 's3://cnk-datalake/experimentos001/athena_datawrangler/')
    # change timezone from UTC to Bogota
    Athenadf['creation_date'] = [(datetime.strptime(str(x)[0:18], '%Y-%m-%d %H:%M:%S') - timedelta(hours=5, minutes=0)).strftime("%Y-%m-%d %H:%M:%S") for x in Athenadf['creation_date']]
    Athenadf['creation_date'] = pd.to_datetime(Athenadf.creation_date)
    # sort values using creation date
    Athenadf = Athenadf.sort_values(by=['creation_date'])
    
    # information store list
    empty_list = []
    # for each piggybank id
    for i,z in zip(temp.piggy_bank_id.tolist(),temp.service_order_created_date.tolist()):
        # define start date and end date
        start_date = temp[(temp.piggy_bank_id == i) & (temp.service_order_created_date == z)].shift_date.iloc[0]
        end_date = temp[(temp.piggy_bank_id == i) & (temp.service_order_created_date == z)].service_order_created_date.iloc[0]
        # if there is no start date (first time)
        if pd.isnull(start_date): 
            # sum amount until end date
            target_df = Athenadf[(Athenadf.piggybank_id == i) & (Athenadf.creation_date <= end_date)]
            empty_list.append([i, target_df.amount.sum(),z])
        else:
            # sum amount until end date and after start date
            target_df = Athenadf[(Athenadf.piggybank_id == i) & (Athenadf.creation_date <= end_date) & (Athenadf.creation_date > start_date)]
            empty_list.append([i, target_df.amount.sum(),z])
    empty_list
    
    # convert store list to dataframe
    core_amount = pd.DataFrame(empty_list, columns=['piggy_bank_id', 'core_amount','service_order_created_date'])
    # add maplocation name
    core_amount = pd.merge(core_amount, temp[['piggy_bank_id', 'maplocation_correct','service_order_created_date']], how='left', on=['piggy_bank_id','service_order_created_date'])
    # drop piggybank id
    core_amount = core_amount.drop(columns=['piggy_bank_id'])
    # rename columns
    core_amount.columns = ['core_amount','service_order_created_date', 'maplocation']
    
    # add data to table 1
    data_merged_inner = pd.merge(data_merged_inner, core_amount, on =['maplocation','service_order_created_date'], how='left')
    data_merged_inner['core_amount'] = pd.to_numeric(data_merged_inner['core_amount'])
    data_merged_inner['value_difference_core_vs_arqueo'] = data_merged_inner['core_amount'] - data_merged_inner['monto_arqueo']
    
    # load data to s3
    datos=data_merged_inner.apply(lambda x: funct_load_data(data_inner=list(x),data_columns=data_merged_inner.columns.tolist()),axis=1)
    data_merged_inner_1=pd.DataFrame(data=[datos[i] for i in datos.index],columns=data_merged_inner.columns.tolist())
    
    ## table with the registers that have a difference of more than 10k
    table_1=data_merged_inner_1[["empresa","ciudad","fecha","maplocation",'core_amount',"monto_arqueo","monto_back","value_difference_back_arqueo",'value_difference_core_vs_arqueo',"service_order_sequence","piggy_bank_operational_account"]]


NameError: name 'funct_load_data' is not defined

In [63]:
## table with registers from the form that had no match in the back
table_2=data_wo_col[(data_wo_col["piggy_bank_operational_account"].isna()) & (data_wo_col["merged"]==0)][['nombre_operario', 'empresa', 'ciudad', 'maplocation', 'fecha','monto_arqueo', 'comentarios', 'service_order_sequence']]

## table with registers on the back that have no match in forms
table_3=data_merged[data_wo_col["monto_arqueo"].isna()][["maplocation","service_order_id","service_order_sequence","service_order_created_date","monto_back","piggy_bank_operational_account"]]

In [64]:
## consolidate the database and save it in the temporal
final_report=pd.concat([table_1,table_2,table_3],axis=0)
final_report.to_excel('/tmp/reporte.xlsx')

In [65]:
## adding the number of days since the service_order.
table_3['Dias']=[i.days for i in datetime.today()-table_3['service_order_created_date']]
table_3.sort_values('Dias',inplace=True,ascending=False)